# <font color='red'>1. Implementacion los algoritmos de extraccion de caracteristicas.</font>

### Algortimo de extraccion de caractristicas 1:

In [ ]:
import numpy as np
import cv2

In [ ]:
def extraccionCaracteristicas1(imagen, N):
    
    # Como son 8 vecinos sin contar el pixel central, la ventana debe ser de 3*3
    tamVentana = 3
    mitad = tamVentana // 2 
    alto = imagen.shape[0]
    ancho = imagen.shape[1]
    vector_caracteristicas = []
    
    # Recorro imagen para crear celdas
    for i in range(0, alto - N + 1, N):  
        for j in range(0, ancho - N + 1, N):
            celda = imagen[i:i+N, j:j+N]
            celdaAmpliada = cv2.copyMakeBorder(celda, mitad, mitad, mitad, mitad, cv2.BORDER_REPLICATE)
            histograma_celda = []
            transiciones_no_uniformes = 0  
            
            # Recorro celdas para crear vetana de vecinos
            for fil in range(1, N + mitad):
                for col in range(1, N + mitad):
                    ventana = celdaAmpliada[fil-mitad:fil+mitad+1, col-mitad:col+mitad+1]
                    pixelCentral = ventana[mitad, mitad]
                    numeros_binarios = []
                    
                    # Recorro ventana para crear el numeor binario
                    for filVecino in range(len(ventana)):
                        for colVecino in range(len(ventana)):
                            vecino = ventana[filVecino][colVecino]
                            
                            # Si no es el pixel central
                            if filVecino != mitad or colVecino != mitad:
                                if vecino < pixelCentral:
                                    numeros_binarios.append(0)
                                else:
                                    numeros_binarios.append(1)
                                    
                    # Convierto el nuemero binario a decimal
                    numero_decimal = sum(bit << idx for idx, bit in enumerate(reversed(numeros_binarios)))
                    #celda[fil-mitad, col-mitad] = numero_decimal
                    
                    # Cuento transiciones no uniformes
                    transiciones = sum(abs(numeros_binarios[i]-numeros_binarios[i-1]) for i in range(1,8))
                    transiciones += abs(numeros_binarios[7]-numeros_binarios[0])
                    if transiciones > 2:
                        transiciones_no_uniformes += 1
                    histograma_celda.append(numero_decimal if transiciones <= 2 else 58)
                    
            # 60 bins para valores entre 0 y 59
            histograma, _ = np.histogram(histograma_celda, bins=range(60))
            
            # Concateno histogramas
            vector_caracteristicas.extend(histograma)
            
    #print(len(vector_caracteristicas)) 
    
    return vector_caracteristicas

### Algortimo de extraccion de caractristicas 2:

In [ ]:
def extraccionCaracteristicas2(imagen, N, M):
    alto = imagen.shape[0]
    ancho = imagen.shape[1]
    histogramas = []
    
    mascara_x = np.array([[-1], [0], [1]])
    mascara_y = np.array([[-1, 0, 1]])
    
    # Convolucion mascaras derivada
    gradiente_x = filtro(imagen, mascara_x)
    gradiente_y = filtro(imagen, mascara_y)

    magnitud = np.sqrt(gradiente_x**2 + gradiente_y**2)
    orientacion = np.rad2deg(np.arctan2(gradiente_y, gradiente_x)) % 180

    bloques = []
    vector_caracteristicas = []
    
    # Recorro imagen para crear celdas
    for i in range(0, alto - N + 1, N): 
        for j in range(0, ancho - N + 1, N):
            celda = imagen[i:i+N, j:j+N]
            histograma = np.zeros(9)
            
            # Recorro celdas para construir los histogramas de cada celda
            for fil in range(N):
                for col in range(N):
                    bin_indice = int(orientacion[i + fil, j + col] % 180 // 20)
                    histograma[bin_indice] += magnitud[i + fil, j + col]
            histogramas.append(histograma)

    # Convierto la lista de histogramas en un array de NumPy
    histogramas = np.array(histogramas).reshape(alto//N, ancho//N, -1)

    # Normalizo los histogramas de los bloques
    for i in range(0, alto//N - M + 1): 
        for j in range(0, ancho//N - M + 1):
            bloque = histogramas[i:i+M, j:j+M]
            bloque = bloque.reshape(-1)  # Convierto el bloque a un vector unidimensional
            norma = np.linalg.norm(bloque)
            bloque = bloque / norma if norma != 0 else bloque  # Normalizo el bloque
            vector_caracteristicas.extend(bloque)
            
    #print(len(vector_caracteristicas))
    
    return vector_caracteristicas

def filtro(imagen, mascara):
    imagen_f = np.zeros(imagen.shape)
    altura, anchura = imagen.shape
    mitad = 3//2 
    imagenAmpliada = cv2.copyMakeBorder(imagen, mitad, mitad, mitad, mitad, cv2.BORDER_REPLICATE)
    for i in range(mitad, altura + mitad):
        for j in range(mitad, anchura + mitad):
            region = imagenAmpliada[i - mitad : i + mitad + 1, j - mitad : j + mitad + 1]
            convolucion = np.sum(np.multiply(mascara, region))
            imagen_f[i - mitad, j - mitad] = convolucion
    return imagen_f

# <font color='red'>2. Cargado de el conjunto de datos.</font>

In [ ]:
import os
from skimage import io

def cargar_imagenes(ruta_carpeta):
    lista_imagenes = []
    etiquetas = []
    
    # Recorro las carpetas en la ruta proporcionada
    for raiz, directorios, archivos in os.walk(ruta_carpeta):
        for nombre_directorio in directorios:
            etiqueta = nombre_directorio  # El nombre de la carpeta es la etiqueta
            ruta_etiqueta = os.path.join(ruta_carpeta, nombre_directorio)
            
            # Recorro los archivos dentro de cada carpeta (etiqueta)
            for raiz2, directorios2, archivos2 in os.walk(ruta_etiqueta):
                for nombre_archivo in archivos2:
                    # Verfico si el archivo es una imagen
                    if nombre_archivo.endswith(".jpg") or nombre_archivo.endswith(".png") or nombre_archivo.endswith(".jpeg"):
                        ruta_imagen = os.path.join(raiz2, nombre_archivo)
                        imagen = io.imread(ruta_imagen)
                        lista_imagenes.append(imagen)
                        etiquetas.append(etiqueta)
    
    return lista_imagenes, etiquetas


carpeta_entrenamiento = 'dataset/train_500'
carpeta_validacion = 'dataset/train_100'
carpeta_prueba = 'dataset/test'

imagenes_entrenamiento, etiquetas_entrenamiento = cargar_imagenes(carpeta_entrenamiento)
imagenes_validacion, etiquetas_validacion = cargar_imagenes(carpeta_validacion)
imagenes_prueba, etiquetas_prueba = cargar_imagenes(carpeta_prueba)

# Junto conjuntos de entrenamiento y validación ya que gridsearch hara la validacion cruzada
imagenes_entrenamiento.extend(imagenes_validacion)
etiquetas_entrenamiento.extend(etiquetas_validacion)

cantidad_imagenes_entrenamiento = len(imagenes_entrenamiento)
cantidad_imagenes_prueba = len(imagenes_prueba)

print(f"Recuento de imágenes en el conjunto de entrenamiento: {cantidad_imagenes_entrenamiento}")
print(f"Recuento de imágenes en el conjunto de prueba: {cantidad_imagenes_prueba}")

# Imprimo la primera imagen y su etiqueta del conjunto de entrenamiento
if len(imagenes_entrenamiento) > 0:
    print("Etiqueta:", etiquetas_entrenamiento[0])
    io.imshow(imagenes_entrenamiento[0])
    io.show()
else:
    print("No se encontraron imágenes de entrenamiento.")

# Imprimo la primera imagen y su etiqueta del conjunto de prueba
if len(imagenes_prueba) > 0:
    print("Etiqueta:", etiquetas_prueba[0])
    io.imshow(imagenes_prueba[0])
    io.show()
else:
    print("No se encontraron imágenes de prueba.")

# <font color='red'>3. Preprocesamiento de los conjuntos de imagenes.</font>

In [ ]:
from skimage import transform, color

def preprocesar_imagenes(imagenes, nuevo_tamaño=(128, 128)):
    imagenes_preprocesadas = []
    contador = 0
    for img in imagenes:
        img_gris = color.rgb2gray(img)
        img_redimensionada = transform.resize(img_gris, nuevo_tamaño)
        imagenes_preprocesadas.append(img_redimensionada)
        if contador is not None:
            print(f"Imagen procesada {contador}")
            contador += 1
    return imagenes_preprocesadas

# Preprocesamiento de las imágenes de entrenamiento
imagenes_entrenamiento_procesadas = preprocesar_imagenes(imagenes_entrenamiento)

# Preprocesamiento de las imágenes de prueba
imagenes_prueba_procesadas = preprocesar_imagenes(imagenes_prueba)

In [ ]:
# Imprimo la primera imagen y su etiqueta del conjunto de entrenamiento
if len(imagenes_entrenamiento_procesadas) > 0:
    print("Etiqueta:", etiquetas_entrenamiento[0])
    io.imshow(imagenes_entrenamiento_procesadas[0])
    io.show()
else:
    print("No se encontraron imágenes de entrenamiento.")
     
# Imprimo la primera imagen y su etiqueta del conjunto de test
if len(imagenes_prueba_procesadas) > 0:
    print("Etiqueta:", etiquetas_prueba[0])
    io.imshow(imagenes_prueba_procesadas[0])
    io.show()
else:
    print("No se encontraron imágenes de test.")

# <font color='red'>4. Extraccion de caracteristicas de los conjuntos de datos.</font>

In [ ]:
def extraer_caracteristicas(imagenes_procesadas, caracteristicasV1, caracteristicasV2, N=16, M=2, tipo=""):
    counter = 0
    print(f"Procesando imágenes {tipo}...")
    for img in imagenes_procesadas:
        
        # Extraccion de caracteristicas mediante el algoritmo 1
        caracteristicas_V1 = extraccionCaracteristicas1(img, N=N)
        
        # Extraccion de caracteristicas mediante el algoritmo 2
        caracteristicas_V2 = extraccionCaracteristicas2(img, N=N, M=M)
        
        caracteristicasV1.append(caracteristicas_V1)
        caracteristicasV2.append(caracteristicas_V2)
        if counter is not None:
            print(f"Imagen {tipo} procesada {counter}")
            counter += 1

# Llamo a la función para las imágenes de entrenamiento
caracteristicas_entrenamientoV1 = []
caracteristicas_entrenamientoV2 = []
extraer_caracteristicas(imagenes_entrenamiento_procesadas, caracteristicas_entrenamientoV1, caracteristicas_entrenamientoV2, tipo="de entrenamiento")

# Llamo a la función para las imágenes de prueba
caracteristicas_pruebaV1 = []
caracteristicas_pruebaV2 = []
extraer_caracteristicas(imagenes_prueba_procesadas, caracteristicas_pruebaV1, caracteristicas_pruebaV2, tipo="de prueba")

# <font color='red'>5. Implementacion de los modelos.</font>

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Convierto las  listas de características y etiquetas a matrices numpy para el entrenamiento de los modelos
X_entrenamientoV1 = np.array(caracteristicas_entrenamientoV1)
X_pruebaV1 = np.array(caracteristicas_pruebaV1)
X_entrenamientoV2 = np.array(caracteristicas_entrenamientoV2)
X_pruebaV2 = np.array(caracteristicas_pruebaV2)
y_entrenamiento = np.array(etiquetas_entrenamiento)
y_prueba = np.array(etiquetas_prueba)

### Modelo 1: SVM (Support vector machine)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score

# Creo los clasificadores SVM
clfV1 = svm.SVC(kernel='linear')
clfV2 = svm.SVC(kernel='linear')

# Defino los parámetros a ajustar
parametros_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1 = GridSearchCV(clfV1, parametros_grid, cv=5, scoring='accuracy')
grid_searchV1.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2 = GridSearchCV(clfV2, parametros_grid, cv=5, scoring='accuracy')
grid_searchV2.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo los mejores clasificadores y sus parámetros
best_clfV1 = grid_searchV1.best_estimator_
best_paramsV1 = grid_searchV1.best_params_

best_clfV2 = grid_searchV2.best_estimator_
best_paramsV2 = grid_searchV2.best_params_

# Realizo predicciones en los conjuntos de prueba con los mejores clasificadores
prediccionesV1 = best_clfV1.predict(X_pruebaV1)
prediccionesV2 = best_clfV2.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
precisionV1 = accuracy_score(y_prueba, prediccionesV1)
precisionV2 = accuracy_score(y_prueba, prediccionesV2)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejores parámetros para V1:", best_paramsV1)
print("Rendimiento de la validación cruzada V1:", grid_searchV1.best_score_)
print("Rendimiento de test V1:", precisionV1)

print("Mejores parámetros para V2:", best_paramsV2)
print("Rendimiento de la validación cruzada V2:", grid_searchV2.best_score_)
print("Rendimiento de test V2:", precisionV2)

### Modelo 2: Regresion logistica

In [ ]:
from sklearn.linear_model import LogisticRegression

# Parámetros para la búsqueda de la mejor C en la regresión logística
param_grid_logistic = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Creo el modelo de regresión logística
logistic_clfV1 = LogisticRegression(max_iter = 1000)
logistic_clfV2 = LogisticRegression(max_iter = 1000)

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train_logistic = GridSearchCV(logistic_clfV1, param_grid_logistic, cv=5, scoring='accuracy')
grid_searchV1_train_logistic.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train_logistic = GridSearchCV(logistic_clfV2, param_grid_logistic, cv=5, scoring='accuracy')
grid_searchV2_train_logistic.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_logistic_clfV1_train = grid_searchV1_train_logistic.best_estimator_
best_logistic_clfV2_train = grid_searchV2_train_logistic.best_estimator_

# Realizo predicciones en el conjunto de prueba
predictionsV1_test_logistic = best_logistic_clfV1_train.predict(X_pruebaV1)
predictionsV2_test_logistic = best_logistic_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test_logistic = accuracy_score(y_prueba, predictionsV1_test_logistic)
accuracyV2_test_logistic = accuracy_score(y_prueba, predictionsV2_test_logistic)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor C V1:", best_logistic_clfV1_train.C)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train_logistic.best_score_)
print("Rendimiento de test V1:", accuracyV1_test_logistic)

print("Mejor C V2:", best_logistic_clfV2_train.C)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train_logistic.best_score_)
print("Rendimiento de test V2:", accuracyV2_test_logistic)

### Modelo 3: Regresion logistica con caracteristicas polinimicas

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Defino el rango de grados a probar
degrees = [1, 2, 3]

# Parámetros para la búsqueda de la mejor C y grado en la regresión logística polinómica
param_grid_logistic_poly = {'polynomialfeatures__degree': degrees, 'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Creo el modelo de regresión logística con características polinómicas
logistic_clfV1_poly = make_pipeline(PolynomialFeatures(), LogisticRegression(max_iter = 1000))
logistic_clfV2_poly = make_pipeline(PolynomialFeatures(), LogisticRegression(max_iter = 1000))

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train_logistic_poly = GridSearchCV(logistic_clfV1_poly, param_grid_logistic_poly, cv=5, scoring='accuracy')
grid_searchV1_train_logistic_poly.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train_logistic_poly = GridSearchCV(logistic_clfV2_poly, param_grid_logistic_poly, cv=5, scoring='accuracy')
grid_searchV2_train_logistic_poly.fit(X_entrenamientoV2, y_entrenamiento)

# Obtener el mejor clasificador
best_logistic_clfV1_train_poly = grid_searchV1_train_logistic_poly.best_estimator_
best_logistic_clfV2_train_poly = grid_searchV2_train_logistic_poly.best_estimator_

# Realizo predicciones en el conjunto de prueba con los mejores clasificadores
predictionsV1_test_logistic_poly = best_logistic_clfV1_train_poly.predict(X_pruebaV1)
predictionsV2_test_logistic_poly = best_logistic_clfV2_train_poly.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test_logistic_poly = accuracy_score(y_prueba, predictionsV1_test_logistic_poly)
accuracyV2_test_logistic_poly = accuracy_score(y_prueba, predictionsV2_test_logistic_poly)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor grado V1:", best_logistic_clfV1_train_poly.named_steps['polynomialfeatures'].degree)
print("Mejor C V1 (con características polinómicas):", best_logistic_clfV1_train_poly.named_steps['logisticregression'].C)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train_logistic_poly.best_score_)
print("Rendimiento de test V1:", accuracyV1_test_logistic_poly)

print("Mejor grado V2:", best_logistic_clfV2_train_poly.named_steps['polynomialfeatures'].degree)
print("Mejor C V2 (con características polinómicas):", best_logistic_clfV2_train_poly.named_steps['logisticregression'].C)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train_logistic_poly.best_score_)
print("Rendimiento de test V2:", accuracyV2_test_logistic_poly)

### Modelo 4: Redes neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier

# Creo el clasificador de red neuronal
mlp = MLPClassifier(max_iter = 1000)

# Defino los parámetros a ajustar
parametros_grid = {
    'hidden_layer_sizes': [(100,), (50, 50)],
}

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1 = GridSearchCV(mlp, parametros_grid, cv=5, scoring='accuracy')
grid_searchV1.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2 = GridSearchCV(mlp, parametros_grid, cv=5, scoring='accuracy')
grid_searchV2.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador y sus parámetros para V1
best_mlpV1 = grid_searchV1.best_estimator_
best_paramsV1 = grid_searchV1.best_params_

# Obtengo el mejor clasificador y sus parámetros para V2
best_mlpV2 = grid_searchV2.best_estimator_
best_paramsV2 = grid_searchV2.best_params_

# Realizo predicciones en los conjuntos de prueba con los mejores clasificadores
prediccionesV1 = best_mlpV1.predict(X_pruebaV1)
prediccionesV2 = best_mlpV2.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
precisionV1 = accuracy_score(y_prueba, prediccionesV1)
precisionV2 = accuracy_score(y_prueba, prediccionesV2)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejores parámetros para V1:", best_paramsV1)
print("Rendimiento de la validación cruzada V1:", grid_searchV1.best_score_)
print("Rendimiento de test V1:", precisionV1)

print("Mejores parámetros para V2:", best_paramsV2)
print("Rendimiento de la validación cruzada V2:", grid_searchV2.best_score_)
print("Rendimiento de test V2:", precisionV2)

### Modelo 5: Ensembles basados en variación de datos (Bagging)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# Parámetros para la búsqueda de el mejor numero de estimadores
param_grid = {'n_estimators': [10, 50, 100]}

# Creo el clasificador base (Decision Tree)
base_classifier = DecisionTreeClassifier()

# Creo el modelo de BaggingClassifier con GridSearchCV para encontrar la mejor profundidad
bagging_clfV1 = BaggingClassifier(base_estimator=base_classifier, random_state=42)
bagging_clfV2 = BaggingClassifier(base_estimator=base_classifier, random_state=42)

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(bagging_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(bagging_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en los conjuntos de prueba con los mejores clasificadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor configuración V1:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Mejor configuración V2:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

### Modelo 6: Ensembles basados en variación de datos (Boosting: Adaboost)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import numpy as np

# Parámetros para la búsqueda de el mejor numero de estimadores
param_grid = {'n_estimators': [10, 50, 100]}

# Creo el clasificador base (Decision Tree)
base_classifier = DecisionTreeClassifier()

# Crear el modelo de AdaBoostClassifier con GridSearchCV para encontrar la mejor profundidad
adaboost_clfV1 = AdaBoostClassifier(base_estimator=base_classifier, random_state=42)
adaboost_clfV2 = AdaBoostClassifier(base_estimator=base_classifier, random_state=42)

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(adaboost_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(adaboost_clfV2, param_grid, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en los conjuntos de prueba con los mejores clasificadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor configuración V1:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Mejor configuración V2:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

### Modelo 7: Ensembles basados en variación de datos (Random forest)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Parámetros para la búsqueda de el mejor numero de estimadores
param_grid = {'n_estimators': [10, 50, 100]}

# Crear el clasificador base (Random Forest)
base_classifier = RandomForestClassifier()

# Crear el modelo de RandomForestClassifier con GridSearchCV para encontrar la mejor configuración
rf_clfV1 = RandomForestClassifier()
rf_clfV2 = RandomForestClassifier()

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(rf_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(rf_clfV2, param_grid, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en el conjunto de prueba con los mejores clasificadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor configuración V1:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Mejor configuración V2:", grid_searchV2_train.best_params_)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

### Modelo 8: Naive Bayes

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Creo el modelo de clasificador con GridSearchCV para encontrar la mejor configuración
nb_clfV1 = GaussianNB()
nb_clfV2 = GaussianNB()

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(nb_clfV1, param_grid={}, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(nb_clfV2, param_grid={}, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en el conjunto de prueba con los mejores clasificadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

### Modelo 9: Ensembles basados en descomposicion (OVO: One vs One).

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
import numpy as np

# Parámetros para la búsqueda de la mejor configuración
param_grid = {'estimator__C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Creo el clasificador base (SVM)
base_classifier = SVC(kernel='linear')

# Creo el modelo de OneVsOneClassifier con GridSearchCV para encontrar la mejor configuración
ovo_clfV1 = OneVsOneClassifier(base_classifier)
ovo_clfV2 = OneVsOneClassifier(base_classifier)

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(ovo_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(ovo_clfV2, param_grid, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en el conjunto de prueba con los mejores clasficadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor configuración V1:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Mejor configuración V2:", grid_searchV2_train.best_params_)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

### Modelo 10: Ensembles basados en descomposicion (OVA: One vs All)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

# Parámetros para la búsqueda de la mejor configuración
param_grid = {'estimator__C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Creo el clasificador base (SVM)
base_classifier = SVC(kernel='linear')

# Creo el modelo de OneVsRestClassifier con GridSearchCV para encontrar la mejor configuración
ova_clfV1 = OneVsRestClassifier(base_classifier)
ova_clfV2 = OneVsRestClassifier(base_classifier)

# Configuro la búsqueda de parámetros con GridSearchCV para V1
grid_searchV1_train = GridSearchCV(ova_clfV1, param_grid, cv=5, scoring='accuracy')
grid_searchV1_train.fit(X_entrenamientoV1, y_entrenamiento)

# Configuro la búsqueda de parámetros con GridSearchCV para V2
grid_searchV2_train = GridSearchCV(ova_clfV2, param_grid, cv=5, scoring='accuracy')
grid_searchV2_train.fit(X_entrenamientoV2, y_entrenamiento)

# Obtengo el mejor clasificador
best_clfV1_train = grid_searchV1_train.best_estimator_
best_clfV2_train = grid_searchV2_train.best_estimator_

# Realizo predicciones en el conjunto de prueba con los mejores clasificadores
predictionsV1_test = best_clfV1_train.predict(X_pruebaV1)
predictionsV2_test = best_clfV2_train.predict(X_pruebaV2)

# Calculo la precisión de los clasificadores en los conjuntos de prueba
accuracyV1_test = accuracy_score(y_prueba, predictionsV1_test)
accuracyV2_test = accuracy_score(y_prueba, predictionsV2_test)

# Imprimo el rendimiento en la validación cruzada y prueba
print("Mejor configuración V1:", grid_searchV1_train.best_params_)
print("Rendimiento de la validación cruzada V1:", grid_searchV1_train.best_score_)
print("Rendimiento de test V1:", accuracyV1_test)

print("Mejor configuración V2:", grid_searchV2_train.best_params_)
print("Rendimiento de la validación cruzada V2:", grid_searchV2_train.best_score_)
print("Rendimiento de test V2:", accuracyV2_test)

# <font color='red'>6. Resultados/Comparacion de los algoritmos de extraccion y modelos</font>

### Resultados de el algoritmo de extraccion de caracteristicas 1 (V1) 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Rendimiento de los modelos
modelos = ['SVM', 'Regresion logistica', 'Regresion logistica polinomica', 'Redes neuronales',
           'Bagging', 'Boosting', 'Random forest', 'Naive Bayes','OVO', 'OVA']
rendimiento_validacion_cruzada = [0.6591666666666667, 0.6504166666666666, 0.6537499999999999, 0.6691666666666667, 0.6733333333333335, 0.45375000000000004, 0.6529166666666667, 0.5075000000000001, 0.6608333333333334, 0.63125]
rendimiento_prueba = [0.5608333333333333, 0.5916666666666667, 0.5916666666666667, 0.6025, 0.5741666666666667, 0.3925, 0.5666666666666667,  0.5191666666666667, 0.55, 0.5733333333333334]

# Creo un DataFrame con los datos
data = pd.DataFrame({'Modelos': modelos, 'Rendimiento_validacion_cruzada': rendimiento_validacion_cruzada, 'Rendimiento_Prueba': rendimiento_prueba})

# Gráfico de barras con rendimiento de entrenamiento/validacion y prueba para cada modelo
fig, ax = plt.subplots(figsize=(10, 6))
data.plot(x='Modelos', y=['Rendimiento_validacion_cruzada', 'Rendimiento_Prueba'], kind='bar', ax=ax)
plt.title('Rendimiento de Modelos para el algoritmo de extraccion de caracteristicas 1')
plt.xlabel('Modelos')
plt.ylabel('Rendimiento')

# Muevo la leyenda fuera del área del gráfico para que se vean los rendimientos
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

# Añado etiquetas de datos a las barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.4f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=8)

# Guardo la gráfica como una imagen
plt.savefig('rendimiento_modelosV1.png')

# Muestro el gráfico
plt.show()

### Resultados de el algoritmo de extraccion de caracteristicas 2 (V2) 

In [ ]:
# Rendimiento de los modelos
modelos = ['SVM', 'Regresion logistica', 'Regresion logistica polinomica', 'Redes neuronales',
           'Bagging', 'Boosting', 'Random forest', 'Naive Bayes','OVO', 'OVA']
rendimiento_validacion_cruzada = [0.7183333333333334, 0.7258333333333333, 0.7254166666666666, 0.7358333333333333, 0.7291666666666667, 0.5283333333333333, 0.7133333333333333, 0.6433333333333333, 0.7120833333333334, 0.705]
rendimiento_prueba = [0.6341666666666667, 0.6408333333333334, 0.6408333333333334, 0.6358333333333334, 0.6308333333333334, 0.42333333333333334, 0.6475,  0.6583333333333333, 0.6366666666666667, 0.6141666666666666]

# Creo un DataFrame con los datos
data = pd.DataFrame({'Modelos': modelos, 'Rendimiento_validacion_cruzada': rendimiento_validacion_cruzada, 'Rendimiento_Prueba': rendimiento_prueba})

# Gráfico de barras con rendimiento de entrenamiento/validacion y prueba para cada modelo
fig, ax = plt.subplots(figsize=(10, 6))
data.plot(x='Modelos', y=['Rendimiento_validacion_cruzada', 'Rendimiento_Prueba'], kind='bar', ax=ax)
plt.title('Rendimiento de Modelos para el algoritmo de extraccion de caracteristicas 2')
plt.xlabel('Modelos')
plt.ylabel('Rendimiento')

# Muevo la leyenda fuera del área del gráfico para que se vean los rendimientos
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

# Añado etiquetas de datos a las barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.4f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=8)

# Guardo la gráfica como una imagen
plt.savefig('rendimiento_modelosV2.png')

# Muestro el gráfico
plt.show()